In [2]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os
from tqdm.auto import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
import os,sys,gc
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria

%matplotlib inline

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_1633897/2347411816.py:21: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


#### Read in stuff

In [3]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'
#############################################################
path_coor = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/U.nc'
coor = xr.open_dataset(path_coor)
maria_coorpres = [1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,  775.,
        750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,  550.,  525.,
        500.,  475.,  450.,  425.,  400.,  375.,  350.,  325.,  300.,  275.,
        250.,  225.,  200.,  175.,  150.,  125.,  100.,   75.,   50.]
#Function####################################################
import json,pickle
def read_some_azimuth_fields(fileloc=None,fieldname=None):
    dict_name = {}
    for inx,obj in enumerate(fileloc):
        field_read = xr.open_dataset(obj)
        dict_name[fieldname[inx]] = field_read
    return dict_name
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

#### Potential Temperature

In [5]:
def do_theta(originpath=None,coor_pres=coor.pres,savepath=None,number=None,exp=None,TC='haiyan'):
    """
    Derive theta and save to /scratch
    """
    if TC=='haiyan':
        coorpres = coor.pres
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[originpath+'mem'+str(int(number))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])        
        lwcrf_theta = derive_var.do_theta(ctrlvar_dict['T']['T'],coorpres)
        read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_theta',lwcrf_theta)
        del ctrlvar_dict,lwcrf_theta
        gc.collect()
        return None
    elif TC=='maria':
        coorpres = coor_pres
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[glob.glob(originpath+exp+'/azim_T*')[0]],fieldname=['T'])        
        lwcrf_theta = derive_var.do_theta(ctrlvar_dict['T']['T'],coorpres)
        read_and_proc.save_to_pickle(savepath+exp+'_theta',lwcrf_theta)
        del ctrlvar_dict,lwcrf_theta
        gc.collect()
        return None
#azim_T_0-168hr_track1.nc

In [28]:
#a = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])['T']
a = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])['T']
aqv = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_QVAPOR_0-168hr_track1.nc'],fieldname=['QVAPOR'])['QVAPOR']

In [32]:
a = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_T_0-168hr_track1.nc'],fieldname=['T'])['T']
aqv = read_some_azimuth_fields(fileloc=['/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/'+'mem'+str(int(3))+'/azim_QVAPOR_0-168hr_track1.nc'],fieldname=['QVAPOR'])['QVAPOR']
aaaa = derive_var.do_rh(aqv['QVAPOR'],a['T'],coor.pres)

In [6]:
#for i in tqdm(['ncrf_96h']):#'ctl','ncrf_36h','ncrf_60h','lwcrf']):
#    do_theta(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/maria/',coor_pres=maria_coorpres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/',number=None,exp=i,TC='maria')
for i in tqdm(range(1,2)):
    do_theta(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/',coor_pres=coor.pres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/',number=i)

  0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/mem1_theta'

#### Radial/Tangential Velocities

In [4]:
def do_uradvtan(originpath=None,coor_pres=coor.pres,savepath=None,number=None,radius=None,exp=None,TC='haiyan'):
    """
    Derive urad/vtan and save to /scratch
    """
    if TC=='haiyan':
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[originpath+'mem'+str(int(number))+'/azim_U_0-168hr_track1.nc',\
                                                         originpath+'mem'+str(int(number))+'/azim_V_0-168hr_track1.nc'],fieldname=['U','V'])
        r500 = nearest_index(ctrlvar_dict['U']['U'].radius,radius)
        ctrlhUrad,ctrlhVtan,_ = derive_var.ruppert_vtmax_calc(ctrlvar_dict['U']['U'].data,ctrlvar_dict['V']['V'].data,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,None)
        read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_urad',ctrlhUrad)
        read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_vtan',ctrlhVtan)
        del ctrlvar_dict,ctrlhUrad,ctrlhVtan
        gc.collect()
        return None
    elif TC=='maria':
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[glob.glob(originpath+exp+'/azim_U_*')[0],\
                                                         glob.glob(originpath+exp+'/azim_V_*')[0]],fieldname=['U','V'])
        r500 = nearest_index(ctrlvar_dict['U']['U'].radius,radius)
        ctrlhUrad,ctrlhVtan,_ = derive_var.ruppert_vtmax_calc(ctrlvar_dict['U']['U'].data,ctrlvar_dict['V']['V'].data,r500,ctrlvar_dict['U']['U'][:,0,:,:].azmiuth.values,None)
        assert np.asarray(ctrlhUrad).shape[0]==39, 'axis shape error' 
        read_and_proc.save_to_pickle(savepath+exp+'_urad',ctrlhUrad)
        read_and_proc.save_to_pickle(savepath+exp+'_vtan',ctrlhVtan)
        del ctrlvar_dict,ctrlhUrad,ctrlhVtan
        gc.collect()
        return None

In [7]:
import importlib
importlib.reload(derive_var)

<module 'tools.derive_var' from '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/tools/derive_var.py'>

In [5]:
from tqdm import tqdm
for i in tqdm(['ctl','ncrf_36h','ncrf_60h','ncrf_96h','lwcrf']):
    do_uradvtan(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/maria/',coor_pres=maria_coorpres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/',number=None,exp=i,TC='maria',radius=600)
#for i in tqdm(range(1,21)):
#    do_uradvtan(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/',coor_pres=coor.pres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/',number=i,
#               radius=625)

100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:16<00:00, 15.23s/it]


#### Relative Humidity

In [38]:
def do_rh(originpath=None,coor_pres=coor.pres,savepath=None,number=None,exp=None,TC='haiyan'):
    """
    Derive theta and save to /scratch
    """
    if TC=='haiyan':
        coorpres = coor.pres
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[originpath+'mem'+str(int(number))+'/azim_T_0-168hr_track1.nc',\
                                                        originpath+'mem'+str(int(number))+'/azim_QVAPOR_0-168hr_track1.nc'],fieldname=['T','QVAPOR'])        
        lwcrf_theta = derive_var.do_rh(ctrlvar_dict['QVAPOR']['QVAPOR'],ctrlvar_dict['T']['T'],coorpres)
        read_and_proc.save_to_pickle(savepath+'mem'+str(int(number))+'_rh',lwcrf_theta)
        del ctrlvar_dict,lwcrf_theta
        gc.collect()
        return None
    elif TC=='maria':
        coorpres = coor_pres
        ctrlvar_dict = read_some_azimuth_fields(fileloc=[glob.glob(originpath+exp+'/azim_T*')[0],\
                                                        glob.glob(originpath+exp+'/azim_QVAPOR*')[0]],fieldname=['T','QVAPOR'])        
        lwcrf_theta = derive_var.do_rh(ctrlvar_dict['QVAPOR']['QVAPOR'],ctrlvar_dict['T']['T'],coorpres)
        read_and_proc.save_to_pickle(savepath+exp+'_rh',lwcrf_theta)
        del ctrlvar_dict,lwcrf_theta
        gc.collect()
        return None

In [41]:
#for i in tqdm(['ncrf_96h']):#'ctl','ncrf_36h','ncrf_60h','lwcrf']):
#    do_theta(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/maria/',coor_pres=maria_coorpres,savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/maria/',number=None,exp=i,TC='maria')
for i in tqdm(range(1,21)):
    do_rh(originpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/',coor_pres=coor.pres,\
          savepath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/testML/output/haiyan/',number=i)

  0%|          | 0/20 [00:00<?, ?it/s]